<a href="https://colab.research.google.com/github/ParalelaUnsaac/2020-2/blob/main/151812_Guia_2_Taxonomia_de_Flynn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

El siguiente código va a permitir que todo código ejecutado en el colab pueda ser medido

In [ ]:
!pip install ipython-autotime

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.02 s (started: 2020-12-31 15:00:13 +00:00)


In [ ]:
print(sum(range(10)))

45
time: 815 µs


Pregunta #1: Que porción de 1 segundo es el valor impreso?

1.   se refiere al simbolo de microsegundos   del orden de 10 elevado a la menos 6
2.   es decir son 880 microsegundos




---



A seguir, tenemos una librería de Python llamado **numba** que realiza paralelización automatica. Asi, se puede verificar que al usar prange() se tiene mejor tiempo de ejecución que al usar range()

In [ ]:
from numba import njit, prange
import numpy as np
A = np.arange(5, 14)
@njit(parallel=True)
def prange_test(A):
    s = 0
    # Without "parallel=True" in the jit-decorator
    # the prange statement is equivalent to range
    for i in prange(A.shape[0]):
        s += A[i]
    return s

print(prange_test(A))

81
time: 2.07 s


In [ ]:
from numba import njit, prange
import numpy as np
A = np.arange(5, 14)
#@njit(parallel=True)
def prange_test(A):
    s = 0
    # Without "parallel=True" in the jit-decorator
    # the prange statement is equivalent to range
    for i in range(A.shape[0]):
        s += A[i]
    return s

print(prange_test(A))



81
time: 7.28 ms


Pregunta #2: identifique otros valores en A, de manera que, serializando, tengamos mejor resultado que paralelizando

*   Primero existen dos formas de ejcutar nuestro codigo el modo objeyo y y el modo no python
*   El modo no python es el mas rapido 
*   El decorador @njit nos dice que se va ejecutar en el modo no python
*   Cuando se agrega la parte paralell lo que hace es paralelizar automaticamente todo el que pueda
*   Una desventaja de esto es que para calculat operaciones o funciones es mas lenta o para una cantidad menor de numeros sim embargo para cantiddades grandes es mas rapia como en iteracions o loops es mucho mas rapida
*   Es por eso que si al areglo A tendria muchos valores con el decorador @njit es mas rapida pero si el arreglo A tendira pocos valores se hace mas lento






---



La Taxonomia de Flynn define 4 tipos de arquitecturas para computación paralela: SISD, SIMD, MISD, y MIMD. 


---


Pregunta #3 : El ultimo código ejecutado es de tipo?

*   Es un SIMD esto porque tienes multiples datos pero para cada uno solo hace una instruccion
por lo tanto seria una sola instruccion y multiples datos






---



Pregunta #4: el siguiente código paralelo es de tipo? Comentar el código para justificar su respuesta


*   Es del tipo SIMD ya que solo se ejecuta una sola instruccion pero habra varios datos la cual es usada en la iteracion



In [ ]:
import threading
import time

def print_time(name, n):
    count = 0   
    print("Para el Hilo: %s, en el momento: %s, su valor de count es: %s" % ( name, time.ctime(), count))
    while count < 5:  #se podria interpretar como los datos ya que para cada uno de ellos hace los mismo
        time.sleep(n)
        count+=1
        print("%s: %s. count %s" % ( name, time.ctime(), count))

        
t1 = threading.Thread(target=print_time, args=("Thread-1", 0, ) )#realiza una soloa instruccion
t2 = threading.Thread(target=print_time, args=("Thread-2", 0, ) )

t1.start()
t2.start()
##por lo tanto es un SIMD

Para el Hilo: Thread-1, en el momento: Thu Dec 10 21:18:06 2020, su valor de count es: 0
Thread-1: Thu Dec 10 21:18:06 2020. count 1
Thread-1: Thu Dec 10 21:18:06 2020. count 2
Thread-1: Thu Dec 10 21:18:06 2020. count 3
Thread-1: Thu Dec 10 21:18:06 2020. count 4
Thread-1: Thu Dec 10 21:18:06 2020. count 5
Para el Hilo: Thread-2, en el momento: Thu Dec 10 21:18:06 2020, su valor de count es: 0
Thread-2: Thu Dec 10 21:18:06 2020. count 1
Thread-2: Thu Dec 10 21:18:06 2020. count 2
Thread-2: Thu Dec 10 21:18:06 2020. count 3
Thread-2: Thu Dec 10 21:18:06 2020. count 4
Thread-2: Thu Dec 10 21:18:06 2020. count 5
time: 13.2 ms




---



Una computadora paralela tipo MIMD es utilizado más en la computación distribuida, ejm. Clusters. El siguiente código en python desktop muestra tal funcionamiento

In [ ]:
#greeting-server.py
!!pip install Pyro4
import Pyro4

@Pyro4.expose   #decorador para exponer las clases, metodos y propiedades de Pyro4
class GreetingMaker(object):    #se crea un clase que se encarga del saludo
    def get_fortune(self, name):  #realiza el saludo a un cliente determinado
        return "Hello, {0}. Here is your fortune message:\n" \
               "Behold the warranty -- the bold print giveth and the fine print taketh away.".format(name)

daemon = Pyro4.Daemon()                # make a Pyro daemon
uri = daemon.register(GreetingMaker)   # register the greeting maker as a Pyro object

print("Ready. Object uri =", uri)      # print the uri so we can use it in the client later
daemon.requestLoop()                   # start the event loop of the server to wait for calls

Ready. Object uri = PYRO:obj_bd19930c45cb4e41b6c6553e8f26e329@localhost:40295
time: 4min 41s


In [ ]:
#greeting-client.py
import Pyro4

uri = input("What is the Pyro uri of the greeting object? ").strip()  #nos pide el uri ya generado
name = input("What is your name? ").strip()   

#realiza el saludo
greeting_maker = Pyro4.Proxy(uri)         # get a Pyro proxy to the greeting object
print(greeting_maker.get_fortune(name))   # call method normally

Pregunta #5: Explique que hace este código de tipo MIMD


*   La primera parte del codigo nos indica que se crea la clase para el saludo y se crea un uri posteriormente
*   A este pueden acceder multiples clientes al mismo tiempo entonces hay muchos datos para los cuales tambien habra multiples instrucciones y todo esto ocurrira al mismo tiempo es decir estan parallelizados
*   Si ejecutamos la primera parte se creara el url para un determinado grupo al cual podran acceder multiples clientes al mismo tiempo
*   Asi sucesivamente se crean mulptiples uri
*   La seguna parte ejecutara el saludo don el repectivo uri si no es el corecto dara un error
*   Es asi como sucede todo el proceso




---



Ejercicio Propuesto: Crear un ejemplo que muestre una computación paralela de tipo MISD



Se puede usar la arquitectura MISD en procesadore SPU(Structure Processing Unit)

*   Si se requiere determinar si un entero positivo dado n es primo. La solución obvia a este problema es probar todos los posibles divisores de n: si ninguno de estos logra dividir n, entonces se dice que n es primo; de lo contrario, se dice que n es compuesto. 
Podemos implementar esta solución como un algoritmo paralelo en una computadora MISD.
*   La idea es dividir el trabajo de probar divisores entre procesadores. Suponemos que hay tantos procesadores en la computadora paralela como divisores de n. Todos los procesadores toman n como entrada, luego cada uno intenta dividirlo por su divisor asociado y emite una salida apropiada basada en el resultado. 
*   Por tanto, es posible determinar en un paso si n es primo. De manera más realista, si hay menos procesadores que divisores potenciales, entonces a cada procesador se le puede dar el trabajo de probar un subconjunto diferente de estos divisores. 
En cualquier caso, se obtiene una aceleración sustancial sobre una implementación puramente secuencial. 
*   Aunque existen soluciones más eficientes para el problema de las pruebas de primalidad, elegi la simple ya que ilustra el punto sin necesidad de mucha sofisticación matemática.





In [ ]:
from numba import njit, prange
import numpy as np
A = np.arange(1, 100000002)
@njit(parallel=True)
def prange_test(A,n):
    s = 0
    # Without "parallel=True" in the jit-decorator
    # the prange statement is equivalent to range
    for i in prange(A.shape[0]):
      if n%A[i]==0:   #si es divisible contamos como un divisor mas
        s += 1
    if s==2:print(n,"Es primo")   #si es el numero de divisores es 2 quiere decir que es un primo
    else :print(n,"No es primo")

n=78888837          #probamos con un numero grande
prange_test(A,n)


78888837 No es primo


In [ ]:
from numba import njit, prange
import numpy as np
A = np.arange(1, 100000002)
#@njit(parallel=True)
def prange_test(A,n):
    s = 0
    # Without "parallel=True" in the jit-decorator
    # the prange statement is equivalent to range
    for i in range(A.shape[0]):
      if n%A[i]==0:     #si es divisible contamos como un divisor mas
        s += 1
    if s==2:print(n,"Es primo") #si es el numero de divisores es 2 quiere decir que es un primo
    else :print(n,"No es primo")
n=78888837          #probamos con un numero grande
prange_test(A,n)

78888837 No es primo
time: 54 s


*   Entonces el primer bloque de codigo es mas veloz comparado con el segundo ya que el primero paraleliza para cada posible divisor de n 
*   El segundo bloque toma 54 segundos de ejecucion compara con el primero que solo le toma 968ms
*   Este es un simple y claro ejemplo de primalidad en un computador MISD esto porque para un solo dato en este caso n se puede crear muchas instrucciones o unidades de procesamiento

Link de muchas otras aplicaiones o algoritmos en el que se puede usar una determinada arquitectura:

http://research.iaun.ac.ir/pd/saeed-nasri/pdfs/UploadFile_9325.pdf


**Referencias**

https://wiki.python.org/moin/ParallelProcessing

https://numba.readthedocs.io/en/stable/user/parallel.html

https://ao.gl/how-to-measure-execution-time-in-google-colab/

http://noisymime.org/blogimages/SIMD.pdf